In [ ]:
#run to start
curr_dir = f'/user_data/csimmon2/git_repos/ptoc'

import sys
sys.path.insert(0,curr_dir)
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
import scipy.stats as stats
import scipy
import statsmodels.api as s
from sklearn import metrics

import os
import pandas as pd
import numpy as np
from nilearn import image, maskers, plotting, datasets
from nilearn.maskers import NiftiMasker
from nilearn.datasets import load_mni152_brain_mask
from nilearn.glm.first_level import compute_regressor
import nibabel as nib
import sys
import time
import itertools 
import warnings


import pdb
import ptoc_params as params

from plotnine import *
#from plotnine import ggplot, aes, geom_point

#hide warnings
import warnings
warnings.filterwarnings('ignore')

#load additional libraries
from nilearn import image, plotting, input_data, glm
from nilearn.input_data import NiftiMasker
import nibabel as nib
import statsmodels.api as sm
from nilearn.datasets import load_mni152_brain_mask, load_mni152_template
from nilearn.glm.first_level import compute_regressor ##?????

data_dir = params.data_dir
results_dir = params.results_dir
fig_dir = params.fig_dir
raw_dir = params.raw_dir
sub_info = params.sub_info
task_info = params.task_info

suf = params.suf
#mni = load_mni152_brain_mask()

'''exp info'''
#load subject info
sub_info = pd.read_csv(f'{curr_dir}/sub_info.csv')
#subs = sub_info[sub_info['group'] == 'control']['sub'].tolist()


##Just controls
#subs = sub_info[sub_info['group'] == 'control']['sub'].tolist()
study = 'ptoc'
study_dir = f"/lab_data/behrmannlab/vlad/{study}"
results_dir = '/user_data/csimmon2/GitHub_Repos/ptoc/results'
exp = ''
#rois = ['LO']  # Run for one ROI initially
#rois = ['LO', 'PFS', 'pIPS','aIPS', 'V1']
control_tasks = ['loc']
file_suf = ''

'''scan params'''
tr = 2 #ptoc_params
vols = 184 #ptoc_params

whole_brain_mask = load_mni152_brain_mask()
mni = load_mni152_template()
brain_masker = NiftiMasker(whole_brain_mask, smoothing_fwhm=0, standardize=True)

In [ ]:
#Extract_roi_coords by hemi

import os
import pandas as pd
import numpy as np
from nilearn import image, plotting
import matplotlib.pyplot as plt

def extract_roi_coords():
    parcels = ['pIPS', 'LO']
    hemispheres = ['left', 'right']
    sub_info = pd.read_csv(os.path.join(curr_dir, 'sub_info.csv'))
    subs = sub_info[sub_info['group'] == 'control']['sub'].tolist()
    #subs = ['sub-025']
    runs = [1, 2, 3]
    run_combos = [(r1, r2) for r1 in runs for r2 in runs if r1 < r2]

    for ss in subs:
        print(f'Processing subject: {ss}')
        sub_dir = os.path.join(study_dir, ss, 'ses-01')
        roi_dir = os.path.join(sub_dir, 'derivatives', 'rois')
        parcel_dir = os.path.join(roi_dir, 'parcels')
        temp_dir = os.path.join(raw_dir, ss, 'ses-01', 'derivatives', 'fsl', 'loc')
        os.makedirs(os.path.join(roi_dir, 'spheres'), exist_ok=True)

        roi_coords = []
        for idx, rc in enumerate(run_combos):
            print(f"Run combination: {rc}")
            all_runs = []
            for rn in rc:
                curr_run_path = os.path.join(temp_dir, f'run-0{rn}', '1stLevel.feat', 'stats', 'zstat3_anat.nii.gz')
                if os.path.exists(curr_run_path):
                    curr_run = image.load_img(curr_run_path)
                    all_runs.append(curr_run)
                else:
                    print(f'File does not exist: {curr_run_path}')
            
            if len(all_runs) != 2:
                print(f"Skipping combination {rc} due to missing data")
                continue

            mean_zstat = image.mean_img(all_runs)

            sub_anat_path = os.path.join(temp_dir, f'run-0{rc[0]}', '1stLevel.feat', 'example_func.nii.gz')
            if not os.path.exists(sub_anat_path):
                print(f"Anatomical image not found: {sub_anat_path}")
                continue
            sub_anat = image.load_img(sub_anat_path)

            for pr in parcels:
                roi_path = os.path.join(parcel_dir, f'{pr}.nii.gz')
                if os.path.exists(roi_path):
                    roi_img = image.load_img(roi_path)
                    roi_data = roi_img.get_fdata()
                    zstat_data = mean_zstat.get_fdata()

                    # Get the middle x-coordinate
                    mid_x = roi_data.shape[0] // 2

                    for hemi in hemispheres:
                        # Create hemisphere mask
                        if hemi == 'left':
                            hemi_mask = np.zeros_like(roi_data)
                            hemi_mask[:mid_x, :, :] = 1
                        else:  # right hemisphere
                            hemi_mask = np.zeros_like(roi_data)
                            hemi_mask[mid_x:, :, :] = 1

                        # Apply masks
                        masked_roi = roi_data * hemi_mask
                        masked_zstat = zstat_data * masked_roi

                        # Find peak coordinates
                        peak_idx = np.unravel_index(np.argmax(masked_zstat), masked_zstat.shape)
                        peak_coords = image.coord_transform(*peak_idx, mean_zstat.affine)

                        roi_coords.append({
                            'index': idx,
                            'task': 'loc',
                            'roi': pr,
                            'hemisphere': hemi,
                            'x': peak_coords[0],
                            'y': peak_coords[1],
                            'z': peak_coords[2]
                        })
                        peak_voxel = np.round(peak_coords).astype(int)
                        print(f"ROI: {pr}, Hemisphere: {hemi}, Run combination: {rc}")
                        print(f"Peak coordinates: {peak_coords}")
                else:
                    print(f'ROI file does not exist: {roi_path}')

        roi_coords_df = pd.DataFrame(roi_coords)
        roi_coords_df.to_csv(os.path.join(roi_dir, 'spheres', 'sphere_coords_hemisphere.csv'), index=False)
        print(f"ROI coordinates saved to {os.path.join(roi_dir, 'spheres', 'sphere_coords_hemisphere.csv')}")

if __name__ == "__main__":
    extract_roi_coords()
    print("ROI coordinate extraction completed.")

In [ ]:
#8.8.24 extract_roi_coords for subject specific rois

import os
import pandas as pd
import numpy as np
from nilearn import image, plotting
import matplotlib.pyplot as plt

def extract_roi_coords():
    parcels = ['pIPS', 'LO']
    sub_info = pd.read_csv(os.path.join(curr_dir, 'sub_info.csv'))
    subs = sub_info[sub_info['group'] == 'control']['sub'].tolist()
    #subs = ['sub-025']
    runs = [1, 2, 3]
    run_combos = [(r1, r2) for r1 in runs for r2 in runs if r1 < r2]
    
    for ss in subs:
        print(f'Processing subject: {ss}')
        sub_dir = os.path.join(study_dir, ss, 'ses-01')
        roi_dir = os.path.join(sub_dir, 'derivatives', 'rois')
        parcel_dir = os.path.join(roi_dir, 'parcels')
        temp_dir = os.path.join(raw_dir, ss, 'ses-01', 'derivatives', 'fsl', 'loc')
        os.makedirs(os.path.join(roi_dir, 'spheres'), exist_ok=True)
        roi_coords = []
        
        for idx, rc in enumerate(run_combos):
            print(f"Run combination: {rc}")
            all_runs = []
            for rn in rc:
                curr_run_path = os.path.join(temp_dir, f'run-0{rn}', '1stLevel.feat', 'stats', 'zstat3_anat.nii.gz')
                if os.path.exists(curr_run_path):
                    curr_run = image.load_img(curr_run_path)
                    all_runs.append(curr_run)
                else:
                    print(f'File does not exist: {curr_run_path}')
            
            if len(all_runs) != 2:
                print(f"Skipping combination {rc} due to missing data")
                continue
            
            mean_zstat = image.mean_img(all_runs)
            
            # Load a single volume for visualization
            sub_anat_path = os.path.join(temp_dir, f'run-0{rc[0]}', '1stLevel.feat', 'example_func.nii.gz')
            if not os.path.exists(sub_anat_path):
                print(f"Anatomical image not found: {sub_anat_path}")
                continue
            sub_anat = image.load_img(sub_anat_path)
            
            for pr in parcels:
                roi_path = os.path.join(parcel_dir, f'{pr}.nii.gz')
                if os.path.exists(roi_path):
                    roi_img = image.load_img(roi_path)
                    masked_data = image.math_img('img1 * img2', img1=mean_zstat, img2=roi_img).get_fdata()
                    peak_idx = np.unravel_index(np.argmax(masked_data), masked_data.shape)
                    peak_coords = image.coord_transform(*peak_idx, mean_zstat.affine)
                    
                    roi_coords.append({
                        'index': idx,  # Add index corresponding to run combination
                        'task': 'loc',
                        'roi': pr,
                        'x': peak_coords[0],
                        'y': peak_coords[1],
                        'z': peak_coords[2]
                    })
                    
                    # After finding peak_coords
                    peak_voxel = np.round(peak_coords).astype(int)

                    print(f"ROI: {pr}, Run combination: {rc}")
                    print(f"Peak coordinates: {peak_coords}")
                else:
                    print(f'ROI file does not exist: {roi_path}')
        
        roi_coords_df = pd.DataFrame(roi_coords)
        roi_coords_df.to_csv(os.path.join(roi_dir, 'spheres', 'sphere_coords.csv'), index=False)
        print(f"ROI coordinates saved to {os.path.join(roi_dir, 'spheres', 'sphere_coords.csv')}")

if __name__ == "__main__":
    extract_roi_coords()
    print("ROI coordinate extraction completed.")

In [ ]:
#8/7/24 at 1:30 pm reviewed and confirmed # 8/6/24 at 10:30 pm Extract_roi_coords
# it runs well and looks good but some of the coordinate are too similar i.e. identical for different runs, this wasn't a problem and confirmed in the below cell. 


import os
import pandas as pd
import numpy as np
from nilearn import image, plotting
import matplotlib.pyplot as plt
import itertools

# Set up directories and parameters
study = 'ptoc'
study_dir = f"/lab_data/behrmannlab/vlad/{study}"
curr_dir = f'/user_data/csimmon2/git_repos/ptoc'
mni_parcel_dir = f'{curr_dir}/roiParcels'

def extract_roi_coords():
    parcels = ['pIPS', 'LO']
    sub_info = pd.read_csv(f'{curr_dir}/sub_info.csv')
    subs = sub_info[sub_info['group'] == 'control']['sub'].tolist()
    #subs = ['sub-025']
    runs = [1, 2, 3]
    run_combos = list(itertools.combinations(runs, 2))
    
    # Load MNI152 template for visualization
    mni_template = image.load_img('/opt/fsl/6.0.3/data/standard/MNI152_T1_2mm_brain.nii.gz')
    
    for ss in subs:
        print(f'Processing subject: {ss}')
        sub_dir = f'{study_dir}/{ss}/ses-01'
        roi_dir = f'{sub_dir}/derivatives/rois'
        os.makedirs(f'{roi_dir}/spheres', exist_ok=True)
        exp_dir = f'{sub_dir}/derivatives/fsl'
        roi_coords = pd.DataFrame(columns=['index', 'task', 'roi', 'x', 'y', 'z'])
        
        index = 0
        for rc in run_combos:
            print(f"Run combination: {rc}")
            all_runs = []
            for rn in rc:
                curr_run_path = f'/lab_data/behrmannlab/vlad/hemispace/{ss}/ses-01/derivatives/fsl/loc/run-0{rn}/1stLevel.feat/stats/zstat3_mni.nii.gz'
                if os.path.exists(curr_run_path):
                    curr_run = image.load_img(curr_run_path)
                    all_runs.append(curr_run)
                else:
                    print(f'File does not exist: {curr_run_path}')
            
            if len(all_runs) != 2:
                print(f"Skipping combination {rc} due to missing data")
                continue
            
            mean_zstat = image.mean_img(all_runs)
            
            for pr in parcels:
                roi_path = f'{mni_parcel_dir}/{pr}.nii.gz'
                if os.path.exists(roi_path):
                    roi = image.load_img(roi_path)
                    # Find peak voxel within ROI
                    masked_data = image.math_img('img1 * img2', img1=mean_zstat, img2=roi).get_fdata()
                    peak_idx = np.unravel_index(np.argmax(masked_data), masked_data.shape)
                    peak_coords = image.coord_transform(*peak_idx, mean_zstat.affine)
                    
                    roi_coords = pd.concat([roi_coords, pd.DataFrame({
                        'index': [index],
                        'task': ['loc'],
                        'roi': [pr],
                        'x': [peak_coords[0]],
                        'y': [peak_coords[1]],
                        'z': [peak_coords[2]]
                    })], ignore_index=True)
                    
                    # Visualize ROI and peak voxel on MNI template
                    display = plotting.plot_roi(roi, bg_img=mni_template,
                                                title=f'{ss} - {pr} - Runs {rc[0]}{rc[1]}')
                    display.add_markers([peak_coords], marker_color='r', marker_size=100)
                    #plt.savefig(f'{roi_dir}/{ss}_{pr}_runs{rc[0]}{rc[1]}_peak_mni.png')
                    plt.close()
                    
                    print(f"ROI: {pr}, Peak coordinates: {peak_coords}")
                else:
                    print(f'ROI file does not exist: {roi_path}')
            
            index += 1
        
        roi_coords.to_csv(f'{roi_dir}/spheres/sphere_coords.csv', index=False)
        print(f"ROI coordinates saved to {roi_dir}/spheres/sphere_coords.csv")

if __name__ == "__main__":
    extract_roi_coords()
    print("ROI coordinate extraction completed.")

In [ ]:
#8.7.24 1 pm, plots heat map of averaged and raw individual run's ROIs

import os
import pandas as pd
import numpy as np
from nilearn import image, plotting
import matplotlib.pyplot as plt
import itertools

# Set up directories and parameters
study = 'hemispace'
study_dir = f"/lab_data/behrmannlab/vlad/{study}"
curr_dir = f'/user_data/csimmon2/git_repos/ptoc'
mni_parcel_dir = f'{curr_dir}/roiParcels'

def visualize_roi_activation_raw():
    parcels = ['pIPS', 'LO']
    sub_info = pd.read_csv(f'{curr_dir}/sub_info.csv')
    #subs = sub_info[sub_info['group'] == 'control']['sub'].tolist()
    subs = ['sub-025', 'sub-057', 'sub-059'] # Update this list as needed
    runs = [1, 2, 3]
    run_combos = list(itertools.combinations(runs, 2))

    # Load MNI152 template for visualization
    mni_template = image.load_img('/opt/fsl/6.0.3/data/standard/MNI152_T1_2mm_brain.nii.gz')

    for ss in subs:
        print(f'Visualizing activation for subject: {ss}')
        sub_dir = f'{study_dir}/{ss}/ses-01'
        roi_dir = f'{sub_dir}/derivatives/rois'
        exp_dir = f'{sub_dir}/derivatives/fsl'

        for rc in run_combos:
            print(f"Run combination: {rc}")
            all_runs = []
            for rn in rc:
                curr_run_path = f'{exp_dir}/loc/run-0{rn}/1stLevel.feat/stats/zstat3_mni.nii.gz'
                if os.path.exists(curr_run_path):
                    curr_run = image.load_img(curr_run_path)
                    all_runs.append(curr_run)
                    # Print summary statistics
                    data = curr_run.get_fdata()
                    print(f"Run {rn} stats: mean={np.mean(data):.4f}, max={np.max(data):.4f}, min={np.min(data):.4f}, std={np.std(data):.4f}")
                else:
                    print(f'File does not exist: {curr_run_path}')
            
            if len(all_runs) != 2:
                print(f"Skipping combination {rc} due to missing data")
                continue
            
            # Manual averaging
            mean_data = np.mean([run.get_fdata() for run in all_runs], axis=0)
            mean_zstat = image.new_img_like(all_runs[0], mean_data)
            
            # Print summary statistics of the averaged data
            print(f"Averaged data stats: mean={np.mean(mean_data):.4f}, max={np.max(mean_data):.4f}, min={np.min(mean_data):.4f}, std={np.std(mean_data):.4f}")

            for pr in parcels:
                roi_path = f'{mni_parcel_dir}/{pr}.nii.gz'
                if os.path.exists(roi_path):
                    roi = image.load_img(roi_path)
                    # Create a masked version of the mean_zstat image
                    masked_zstat = image.math_img('img1 * img2', img1=mean_zstat, img2=roi)

                    # Visualize the activation within the ROI
                    fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15, 5))
                    fig.suptitle(f'{ss} - {pr} - Runs {rc[0]}{rc[1]} Activation')

                    # Sagittal view
                    plotting.plot_stat_map(masked_zstat, bg_img=mni_template,
                                           display_mode='x', cut_coords=1,
                                           threshold=0, colorbar=True, axes=ax1)
                    ax1.set_title('Sagittal')

                    # Coronal view
                    plotting.plot_stat_map(masked_zstat, bg_img=mni_template,
                                           display_mode='y', cut_coords=1,
                                           threshold=0, colorbar=True, axes=ax2)
                    ax2.set_title('Coronal')

                    # Axial view
                    plotting.plot_stat_map(masked_zstat, bg_img=mni_template,
                                           display_mode='z', cut_coords=1,
                                           threshold=0, colorbar=True, axes=ax3)
                    ax3.set_title('Axial')

                    plt.tight_layout()
                    #plt.savefig(f'{roi_dir}/{ss}_{pr}_runs{rc[0]}{rc[1]}_activation.png')
                    #plt.close()
                    print(f"Activation map saved for ROI: {pr}")
                else:
                    print(f'ROI file does not exist: {roi_path}')

    print("ROI activation visualization completed.")

def visualize_roi_activation_combos():
    parcels = ['pIPS', 'LO']
    sub_info = pd.read_csv(f'{curr_dir}/sub_info.csv')
    #subs = sub_info[sub_info['group'] == 'control']['sub'].tolist()
    subs = ['sub-025', 'sub-057', 'sub-059']  # Update this list as needed
    
    runs = [1, 2, 3]
    run_combos = list(itertools.combinations(runs, 2))

    # Load MNI152 template for visualization
    mni_template = image.load_img('/opt/fsl/6.0.3/data/standard/MNI152_T1_2mm_brain.nii.gz')

    for ss in subs:
        print(f'Visualizing activation for subject: {ss}')
        sub_dir = f'{study_dir}/{ss}/ses-01'
        roi_dir = f'{sub_dir}/derivatives/rois'
        exp_dir = f'{sub_dir}/derivatives/fsl'

        for rc in run_combos:
            print(f"Run combination: {rc}")
            all_runs = []
            for rn in rc:
                curr_run_path = f'{exp_dir}/loc/run-0{rn}/1stLevel.feat/stats/zstat3_mni.nii.gz'
                if os.path.exists(curr_run_path):
                    curr_run = image.load_img(curr_run_path)
                    all_runs.append(curr_run)
                else:
                    print(f'File does not exist: {curr_run_path}')

            if len(all_runs) != 2:
                print(f"Skipping combination {rc} due to missing data")
                continue

            mean_zstat = image.mean_img(all_runs)

            for pr in parcels:
                roi_path = f'{mni_parcel_dir}/{pr}.nii.gz'
                if os.path.exists(roi_path):
                    roi = image.load_img(roi_path)
                    
                    # Create a masked version of the mean_zstat image
                    masked_zstat = image.math_img('img1 * img2', img1=mean_zstat, img2=roi)
                    
                    # Visualize the activation within the ROI
                    fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15, 5))
                    fig.suptitle(f'{ss} - {pr} - Runs {rc[0]}{rc[1]} Activation')
                    
                    # Sagittal view
                    plotting.plot_stat_map(masked_zstat, bg_img=mni_template, 
                                        display_mode='x', cut_coords=1, 
                                        threshold=0, colorbar=True, axes=ax1)
                    ax1.set_title('Sagittal')

                    # Coronal view
                    plotting.plot_stat_map(masked_zstat, bg_img=mni_template, 
                                        display_mode='y', cut_coords=1, 
                                        threshold=0, colorbar=True, axes=ax2)
                    ax2.set_title('Coronal')

                    # Axial view
                    plotting.plot_stat_map(masked_zstat, bg_img=mni_template, 
                                        display_mode='z', cut_coords=1, 
                                        threshold=0, colorbar=True, axes=ax3)
                    ax3.set_title('Axial')
                    
                    plt.tight_layout()
                    #plt.savefig(f'{roi_dir}/{ss}_{pr}_runs{rc[0]}{rc[1]}_activation.png')
                    #plt.close()

                    print(f"Activation map saved for ROI: {pr}")
                else:
                    print(f'ROI file does not exist: {roi_path}')

    print("ROI activation visualization completed.")

if __name__ == "__main__":
    #visualize_roi_activation_combos()
    #visualize_roi_activation_raw()

In [ ]:
#fc_ppi native space with hemispheres

import os
import pandas as pd
import numpy as np
from nilearn import image, input_data
from nilearn.maskers import NiftiMasker
from nilearn.datasets import load_mni152_brain_mask
from nilearn.glm.first_level import compute_regressor
import nibabel as nib
import sys

# Import your parameters
curr_dir = f'/user_data/csimmon2/git_repos/ptoc'
sys.path.insert(0, curr_dir)
import ptoc_params as params

# Set up directories and parameters
study = 'ptoc'
study_dir = f"/lab_data/behrmannlab/vlad/{study}"
results_dir = '/user_data/csimmon2/git_repos/ptoc/results'
raw_dir = params.raw_dir

sub_info = pd.read_csv(f'{curr_dir}/sub_info.csv')
#subs = sub_info[sub_info['group'] == 'control']['sub'].tolist()
subs = ['sub-068', 'sub-071', 'sub-083', 'sub-084']  # Update this list as needed
rois = ['PFS', 'aIPS'] # Run for one ROI initially
hemispheres = ['left', 'right']
run_num = 3
runs = list(range(1, run_num + 1))
run_combos = [[rn1, rn2] for rn1 in range(1, run_num + 1) for rn2 in range(rn1 + 1, run_num + 1)]

def extract_roi_sphere(img, coords):
    roi_masker = input_data.NiftiSpheresMasker([tuple(coords)], radius=6)
    seed_time_series = roi_masker.fit_transform(img)
    phys = np.mean(seed_time_series, axis=1).reshape(-1, 1)
    return phys

def make_psy_cov(runs, ss):
    temp_dir = f'{raw_dir}/{ss}/ses-01'
    cov_dir = f'{temp_dir}/covs'
    vols, tr = 184, 2.0
    times = np.arange(0, vols * tr, tr)
    full_cov = pd.DataFrame(columns=['onset', 'duration', 'value'])

    for rn in runs:
        ss_num = ss.split('-')[1]
        obj_cov_file = f'{cov_dir}/catloc_{ss_num}_run-0{rn}_Object.txt'
        scr_cov_file = f'{cov_dir}/catloc_{ss_num}_run-0{rn}_Scramble.txt'

        if not os.path.exists(obj_cov_file) or not os.path.exists(scr_cov_file):
            print(f'Covariate file not found for run {rn}')
            continue

        obj_cov = pd.read_csv(obj_cov_file, sep='\t', header=None, names=['onset', 'duration', 'value'])
        scr_cov = pd.read_csv(scr_cov_file, sep='\t', header=None, names=['onset', 'duration', 'value'])
        scr_cov['value'] *= -1
        full_cov = pd.concat([full_cov, obj_cov, scr_cov])

    full_cov = full_cov.sort_values(by=['onset']).reset_index(drop=True)
    cov = full_cov.to_numpy()
    valid_onsets = cov[:, 0] < times[-1]
    cov = cov[valid_onsets]

    if cov.shape[0] == 0:
        print('No valid covariate data after filtering. Returning zeros array.')
        return np.zeros((vols, 1))

    psy, _ = compute_regressor(cov.T, 'spm', times)
    return psy

def conduct_analyses():
    for ss in subs:
        print(f"Processing subject: {ss}")
        sub_dir = f'{study_dir}/{ss}/ses-01/'
        roi_dir = f'{sub_dir}derivatives/rois'
        temp_dir = f'{raw_dir}/{ss}/ses-01/derivatives/fsl/loc'
        
        roi_coords = pd.read_csv(f'{roi_dir}/spheres/sphere_coords_hemisphere.csv')
        
        out_dir = f'{study_dir}/{ss}/ses-01/derivatives'
        os.makedirs(f'{out_dir}/fc', exist_ok=True)
        
        # subject-specific brain mask
        def get_subject_mask(ss):
            mask_path  = f'{raw_dir}/{ss}/ses-01/anat/{ss}_ses-01_T1w_brain_mask.nii.gz'
            return nib.load(mask_path)
        
        # Load subject-specific mask
        whole_brain_mask = get_subject_mask(ss)
        brain_masker = NiftiMasker(whole_brain_mask, smoothing_fwhm=0, standardize=True)

        for tsk in ['loc']:
            for rr in rois:
                for hemi in hemispheres:
                    print(f"Processing ROI: {rr}, Hemisphere: {hemi}")
                    
                    fc_file = f'{out_dir}/fc/{ss}_{rr}_{hemi}_{tsk}_fc.nii.gz'
                    ppi_file = f'{out_dir}/fc/{ss}_{rr}_{hemi}_{tsk}_ppi.nii.gz'
                    
                    do_fc = not os.path.exists(fc_file)
                    do_ppi = not os.path.exists(ppi_file)
                    
                    if not do_fc and not do_ppi:
                        print(f'Both FC and PPI files for {rr} {hemi} already exist. Skipping...')
                        continue
                    
                    all_runs_fc = []
                    all_runs_ppi = []
                    
                    for rcn, rc in enumerate(run_combos):
                        curr_coords = roi_coords[(roi_coords['index'] == rcn) & 
                                                 (roi_coords['task'] == tsk) & 
                                                 (roi_coords['roi'] == rr) &
                                                 (roi_coords['hemisphere'] == hemi)]
                        
                        if curr_coords.empty:
                            print(f"No coordinates found for {rr}, {hemi}, run combo {rc}")
                            continue
                        
                        coords = curr_coords[['x', 'y', 'z']].values.tolist()[0]
                        
                        filtered_list = [image.clean_img(image.load_img(f'{temp_dir}/run-0{rn}/1stLevel.feat/filtered_func_data_reg.nii.gz'), standardize=True) for rn in rc]
                        img4d = image.concat_imgs(filtered_list)
                        
                        phys = extract_roi_sphere(img4d, coords)
                        
                        # Ensure phys length matches the number of volumes
                        if phys.shape[0] > 184 * len(rc):
                            phys = phys[:184 * len(rc)]
                        
                        brain_time_series = brain_masker.fit_transform(img4d)
                        
                        if do_fc:
                            # FC Analysis
                            correlations = np.dot(brain_time_series.T, phys) / phys.shape[0]
                            correlations = np.arctanh(correlations.ravel())
                            correlation_img = brain_masker.inverse_transform(correlations)
                            all_runs_fc.append(correlation_img)
                        
                        if do_ppi:
                            # PPI Analysis
                            psy = make_psy_cov(rc, ss)  # Generate psy for the current run combination
                            
                            # Ensure psy length matches phys
                            if psy.shape[0] > phys.shape[0]:
                                psy = psy[:phys.shape[0]]
                            elif psy.shape[0] < phys.shape[0]:
                                phys = phys[:psy.shape[0]]
                                brain_time_series = brain_time_series[:psy.shape[0]]
                            
                            ppi_regressor = phys * psy
                            ppi_correlations = np.dot(brain_time_series.T, ppi_regressor) / ppi_regressor.shape[0]
                            ppi_correlations = np.arctanh(ppi_correlations.ravel())
                            ppi_img = brain_masker.inverse_transform(ppi_correlations)
                            all_runs_ppi.append(ppi_img)
                    
                    if do_fc:
                        mean_fc = image.mean_img(all_runs_fc)
                        nib.save(mean_fc, fc_file)
                        print(f'Saved FC result for {rr} {hemi}')
                    
                    if do_ppi:
                        mean_ppi = image.mean_img(all_runs_ppi)
                        nib.save(mean_ppi, ppi_file)
                        print(f'Saved PPI result for {rr} {hemi}')

# Call the function
conduct_analyses()

In [ ]:
#fc_ppi all in subject space
import os
import pandas as pd
import numpy as np
from nilearn import image, input_data
from nilearn.maskers import NiftiMasker
from nilearn.datasets import load_mni152_brain_mask
from nilearn.glm.first_level import compute_regressor
import nibabel as nib
import sys


# Import your parameters
curr_dir = f'/user_data/csimmon2/git_repos/ptoc'
sys.path.insert(0, curr_dir)
import ptoc_params as params

# Set up directories and parameters
study = 'ptoc'
study_dir = f"/lab_data/behrmannlab/vlad/{study}"
results_dir = '/user_data/csimmon2/git_repos/ptoc/results'
raw_dir = params.raw_dir

sub_info = pd.read_csv(f'{curr_dir}/sub_info.csv')
#subs = sub_info[sub_info['group'] == 'control']['sub'].tolist()
subs = ['sub-085', 'sub-087', 'sub-088', 'sub-093']  # Update this list as needed
rois = ['LO', 'pIPS']
run_num = 3
runs = list(range(1, run_num + 1))
run_combos = [[rn1, rn2] for rn1 in range(1, run_num + 1) for rn2 in range(rn1 + 1, run_num + 1)]

def extract_roi_sphere(img, coords):
    roi_masker = input_data.NiftiSpheresMasker([tuple(coords)], radius=6)
    seed_time_series = roi_masker.fit_transform(img)
    phys = np.mean(seed_time_series, axis=1).reshape(-1, 1)
    return phys

def make_psy_cov(runs, ss):
    temp_dir = f'{raw_dir}/{ss}/ses-01'
    cov_dir = f'{temp_dir}/covs'
    vols, tr = 184, 2.0
    times = np.arange(0, vols * tr, tr)
    full_cov = pd.DataFrame(columns=['onset', 'duration', 'value'])

    for rn in runs:
        ss_num = ss.split('-')[1]
        obj_cov_file = f'{cov_dir}/catloc_{ss_num}_run-0{rn}_Object.txt'
        scr_cov_file = f'{cov_dir}/catloc_{ss_num}_run-0{rn}_Scramble.txt'

        if not os.path.exists(obj_cov_file) or not os.path.exists(scr_cov_file):
            print(f'Covariate file not found for run {rn}')
            continue

        obj_cov = pd.read_csv(obj_cov_file, sep='\t', header=None, names=['onset', 'duration', 'value'])
        scr_cov = pd.read_csv(scr_cov_file, sep='\t', header=None, names=['onset', 'duration', 'value'])
        scr_cov['value'] *= -1
        full_cov = pd.concat([full_cov, obj_cov, scr_cov])

    full_cov = full_cov.sort_values(by=['onset']).reset_index(drop=True)
    cov = full_cov.to_numpy()
    valid_onsets = cov[:, 0] < times[-1]
    cov = cov[valid_onsets]

    if cov.shape[0] == 0:
        print('No valid covariate data after filtering. Returning zeros array.')
        return np.zeros((vols, 1))

    psy, _ = compute_regressor(cov.T, 'spm', times)
    return psy

def conduct_analyses():
    for ss in subs:
        print(f"Processing subject: {ss}")
        sub_dir = f'{study_dir}/{ss}/ses-01/'
        roi_dir = f'{sub_dir}derivatives/rois'
        temp_dir = f'{raw_dir}/{ss}/ses-01/derivatives/fsl/loc'
        
        roi_coords = pd.read_csv(f'{roi_dir}/spheres/sphere_coords.csv')
        
        out_dir = f'{study_dir}/{ss}/ses-01/derivatives'
        os.makedirs(f'{out_dir}/fc', exist_ok=True)
        os.makedirs(f'{out_dir}/fc', exist_ok=True)
        
        # subject-specific brain mask
        def get_subject_mask(ss):
            mask_path  = f'{raw_dir}/{ss}/ses-01/anat/{ss}_ses-01_T1w_brain_mask.nii.gz'
            return nib.load(mask_path)
        
        # Load subject-specific mask
        whole_brain_mask = get_subject_mask(ss)
        brain_masker = NiftiMasker(whole_brain_mask, smoothing_fwhm=0, standardize=True)

        for tsk in ['loc']:
            for rr in rois:
                print(f"Processing ROI: {rr}")
                
                fc_file = f'{out_dir}/fc/{ss}_{rr}_{tsk}_fc.nii.gz'
                ppi_file = f'{out_dir}/fc/{ss}_{rr}_{tsk}_ppi.nii.gz'
                
                do_fc = not os.path.exists(fc_file)
                do_ppi = not os.path.exists(ppi_file)
                
                if not do_fc and not do_ppi:
                    print(f'Both FC and PPI files for {rr} already exist. Skipping...')
                    continue
                
                all_runs_fc = []
                all_runs_ppi = []
                
                for rcn, rc in enumerate(run_combos):
                    curr_coords = roi_coords[(roi_coords['index'] == rcn) & (roi_coords['task'] == tsk) & (roi_coords['roi'] == rr)]
                    coords = curr_coords[['x', 'y', 'z']].values.tolist()[0]
                    
                    filtered_list = [image.clean_img(image.load_img(f'{temp_dir}/run-0{rn}/1stLevel.feat/filtered_func_data_reg.nii.gz'), standardize=True) for rn in rc]
                    img4d = image.concat_imgs(filtered_list)
                    
                    phys = extract_roi_sphere(img4d, coords)
                    
                    # Ensure phys length matches the number of volumes
                    if phys.shape[0] > 184 * len(rc):
                        phys = phys[:184 * len(rc)]
                    
                    brain_time_series = brain_masker.fit_transform(img4d)
                    
                    if do_fc:
                        # FC Analysis
                        correlations = np.dot(brain_time_series.T, phys) / phys.shape[0]
                        correlations = np.arctanh(correlations.ravel())
                        correlation_img = brain_masker.inverse_transform(correlations)
                        all_runs_fc.append(correlation_img)
                    
                    if do_ppi:
                        # PPI Analysis
                        psy = make_psy_cov(rc, ss)  # Generate psy for the current run combination
                        
                        # Ensure psy length matches phys
                        if psy.shape[0] > phys.shape[0]:
                            psy = psy[:phys.shape[0]]
                        elif psy.shape[0] < phys.shape[0]:
                            phys = phys[:psy.shape[0]]
                            brain_time_series = brain_time_series[:psy.shape[0]]
                        
                        ppi_regressor = phys * psy
                        ppi_correlations = np.dot(brain_time_series.T, ppi_regressor) / ppi_regressor.shape[0]
                        ppi_correlations = np.arctanh(ppi_correlations.ravel())
                        ppi_img = brain_masker.inverse_transform(ppi_correlations)
                        all_runs_ppi.append(ppi_img)
                
                if do_fc:
                    mean_fc = image.mean_img(all_runs_fc)
                    nib.save(mean_fc, fc_file)
                    print(f'Saved FC result for {rr}')
                
                if do_ppi:
                    mean_ppi = image.mean_img(all_runs_ppi)
                    nib.save(mean_ppi, ppi_file)
                    print(f'Saved PPI result for {rr}')

# Call the function
conduct_analyses()

In [ ]:
### conversion is a flirt command and needs to be run in the terminal. ###
## run hemi_subj2parcel.py

In [ ]:
##to plot figures use visualize_fc_ppi.ipynb